In [1]:
# imports
from torch.utils.data import Dataset

from torch import nn

from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
#from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import time
import cv2
import os
import json
from typing import Dict, List
from torchvision.transforms import Normalize

In [2]:
class ObjectDetector(nn.Module):
    def __init__(self, base_model, num_classes) -> None:
        super().__init__()
        self.base_model = base_model
        self.regressor = nn.Sequential(
            nn.Linear(base_model.fc.in_features, 128), 
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(32, 4),
            nn.Sigmoid()
            
        )
        self.classifier = nn.Sequential(
			nn.Linear(base_model.fc.in_features, 512),
			nn.ReLU(),
			nn.Dropout(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Dropout(),
			nn.Linear(512, self.numClasses)
		)
		# set the classifier of our base model to produce outputs
		# from the last convolution block
        self.base_model.fc = nn.Identity()
    def forward(self, x):
        features = self.base_model(x)
        bboxes = self.regressor(x)
        class_logits = self.classifier(x)
        return (bboxes, class_logits)



In [90]:

class ImageDataset(Dataset):
    def __init__(self, annotations_path, images_path, channels, transforms=None) -> None:
        self.transforms = transforms
        self.images: List[torch.Tensor] = []
        self.image_bboxes: List[Dict] = []
        
        annotations_files = os.listdir(annotations_path)
        for json_file_name in annotations_files:
            file_path = annotations_path + json_file_name
            f = open(file_path, 'r')
            annotation: Dict = json.load(f)[0] # as it is a list with one element the dict
            coordinates: Dict = {
                "boxes": []
            }
            
            for bbox in annotation["annotations"]:
                coordinates["boxes"].append(bbox["coordinates"])

            self.image_bboxes.append(coordinates)
            image = cv2.imread(images_path + annotation["image"])
            image: torch.Tensor = torch.tensor(image, dtype=float).permute(1,0,2) # (X, Y, RGB) (W,H,RGB) we do this to match with the bboxes coordinates (x,y)
            image = (image * 2)/255. - 1 # normalization [-1, 1]
            self.images.append(image)
        # Compute the mean of the dataset
        stack = torch.stack(tuple(self.images))
        self.means = []
        self.stds = []
        for channel in range(channels):
            channel_stack: List[torch.Tensor] = stack[:, :, :, channel].reshape(stack.shape[0] * stack.shape[1] * stack.shape[2])
            self.means.append(
                float(channel_stack.mean())
                )
            self.stds.append(
                float(channel_stack.std())
                )

    def __getitem__(self, idx): 
        if self.transforms:
            return (self.transforms(self.images[idx]) , self.image_bboxes[idx])
        else:
            return (self.images[idx], self.image_bboxes[idx])
    def __len__(self):
        return self.images.size(0)



In [91]:
dataset = ImageDataset(
    annotations_path="../datasets/annotations/",
    images_path="../../frames/",
    channels=3
)


: 

: 

In [86]:
float(dataset.means[0])

0.26350067177310516

In [75]:
torch.stack((dataset[0][0], dataset[1][0]))[:, :, :, 0].reshape(1920 * 1080 ).mean()

RuntimeError: shape '[2073600]' is invalid for input of size 4147200

In [60]:
x = torch.Tensor([[[1 ,2, 3 ]], [[3, 4, 4]]])
x[:, :, 0]

tensor([[1.],
        [3.]])